In [2]:
!pwd

/home/infres/amathur-23/DADA/src/mid_fusion


In [3]:
%cd /home/infres/amathur-23/DADA/src

/home/infres/amathur-23/DADA/src


In [9]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['HF_HOME'] = '/data/amathur-23/DADA/transformers'

In [10]:
import torch
import torch.nn as nn

from authorship_attribution.model import AuthorshipLLM
from SpeechCLR.models import SpeechEmbedder

In [11]:
TEXT_MODEL_NAME = 'microsoft/deberta-v3-large'

text_model = AuthorshipLLM(
    model_name=TEXT_MODEL_NAME,
)

config.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/874M [00:00<?, ?B/s]

In [12]:
speech_model = SpeechEmbedder()

/home/infres/amathur-23/DADA/dada/lib/python3.12/site-packages/transformers/configuration_utils.py:306: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
text_model

AuthorshipLLM(
  (model): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 1024, padding_idx=0)
      (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-23): 24 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (key_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (value_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
     

In [14]:
speech_model

SpeechEmbedder(
  (feature_model): Wav2Vec2ForCTC(
    (wav2vec2): Wav2Vec2Model(
      (feature_extractor): Wav2Vec2FeatureEncoder(
        (conv_layers): ModuleList(
          (0): Wav2Vec2LayerNormConvLayer(
            (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
            (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
            (activation): GELUActivation()
          )
          (1-4): 4 x Wav2Vec2LayerNormConvLayer(
            (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
            (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
            (activation): GELUActivation()
          )
          (5-6): 2 x Wav2Vec2LayerNormConvLayer(
            (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
            (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
            (activation): GELUActivation()
          )
        )
      )
      (feature_projection): Wav2Vec2FeatureProjection(
        (l

In [17]:
file = "../datasets/InTheWild/wild_transcription_meta.json"

import pandas as pd
df = pd.read_json(file)

In [18]:
df.head()

,file,content,speaker,label
0,11416.wav,"In fact, dead people.",Donald Trump,bona-fide
1,11424.wav,and the local government's act for whiz.,FDR,bona-fide
2,11422.wav,But I have made clear our trade imbalance is ...,Donald Trump,bona-fide
3,11426.wav,under very difficult circumstances is far gre...,Barack Obama,bona-fide
4,11420.wav,in the modern history of the United States of...,Bernie Sanders,bona-fide


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import os
import librosa
import yaml

from transformers import AutoTokenizer, DebertaV2Tokenizer

SUPPORTED_FORMATS = ["wav", "mp3", "flac"]

from mid_fusion.utils.datasets import load_audio, pad, get_spoof_list

class InTheWildDataset(torch.utils.data.Dataset):
    def __init__(
        self,
        root_dir,
        metadata_file="meta.csv",
        include_spoofs=False,
        bonafide_label="bona-fide",
        filename_col="file",
        transcription_col='content',
        sampling_rate=16000,
        max_duration=4,
        split="train",
        config=None,
        max_text_length = 64,
        mode="classification",
        text_tokenizer_name = None,
    ):
        """
        Args:
            root_dir (str): Root directory of the dataset
            metadata_file (str): Name of the metadata file
            include_spoofs (bool): Whether to include spoofed data
            bonafide_label (str): Label for bonafide data
            sampling_rate (int): Sampling rate of the audio files
            max_duration (int): Maximum duration of the audio files in seconds
            split (str): Split of the dataset (train, val, test)
            config (dict): Configuration dictionary
            mode (str): Mode of the dataset (triplet, classification)
        """

        if text_tokenizer_name is None:
            raise NotImplementedError("Text tokenizer is required for this dataset")
        
        self.text_tokenizer = AutoTokenizer.from_pretrained(text_tokenizer_name)
        self.max_text_length = max_text_length

        self.root_dir = root_dir
        self.metadata_file = metadata_file
        if self.metadata_file.endswith(".csv"):
            self.df = pd.read_csv(os.path.join(root_dir, metadata_file))
        elif self.metadata_file.endswith(".json"):
            self.df = pd.read_json(os.path.join(root_dir, metadata_file))
        self.include_spoofs = include_spoofs
        self.split = split
        self.mode = mode

        self.bonafide_label = bonafide_label

        self.sampling_rate = sampling_rate
        self.max_duration = max_duration
        self.cut = self.sampling_rate * self.max_duration  # padding

        # Filter out spoofed data if include_spoofs is False
        if not self.include_spoofs:
            self.df = self.df[self.df["label"] == bonafide_label]

        # Filter out data based on filename in config['split']
        if config is not None:
            self.config = yaml.safe_load(open(config, "r"))
            self.df = self.df[self.df[filename_col].isin(self.config[split])]

        # Create id to filename mapping and id to label mapping
        ids = np.arange(len(self.df))
        filenames = self.df[filename_col].values
        labels = self.df["label"].values
        authors = self.df["speaker"].values
        transcriptions = self.df[transcription_col].values

        self.id_to_filename = dict(zip(ids, filenames))
        self.id_to_label = dict(zip(ids, labels))
        self.id_to_author = dict(zip(ids, authors))
        self.id_to_transcription = dict(zip(ids, transcriptions))

    def text_to_input_dict(self, text):
        inputs = self.text_tokenizer(
            text,
            return_tensors="pt",
            padding="max_length",
            max_length=self.max_text_length,
            truncation=True,
        )

        inputs = {k: v.squeeze() for k, v in inputs.items()}

        return inputs

    def load_audio_tensor(self, idx):
        # Load audio file
        # print(idx)
        try:
            filename = os.path.join(self.root_dir, self.id_to_filename[idx])
        except Exception as e:
            print(e)
            print(idx)
        audio_arr, _ = load_audio(filename, self.sampling_rate)
        audio_tensor = torch.tensor(pad(audio_arr, self.cut)).float()
        return audio_tensor

    def get_triplets_from_anchor(self, anchor_idx):
        anchor_author = self.id_to_author[anchor_idx]
        positive_id = np.random.choice(
            np.where(self.df["speaker"].values == anchor_author)[0]
        )
        negative_ids = np.where(self.df["speaker"].values != anchor_author)[0]

        return positive_id, np.random.choice(negative_ids)

    def __len__(self):
        return len(self.id_to_filename)

    def __getitem__(self, idx):
        x = self.load_audio_tensor(idx)
        y = int(self.id_to_label[idx] == self.bonafide_label)
        author = self.id_to_author[idx]
        transcription = self.text_to_input_dict(self.id_to_transcription[idx])

        if self.mode == "classification":
            return {"x": x, "label": y, "author": author, "transcription": transcription}

        elif self.mode == "triplet":
            id_p, id_n = self.get_triplets_from_anchor(idx)

            x_p = self.load_audio_tensor(id_p)
            x_n = self.load_audio_tensor(id_n)

            return {"anchor": x, "positive": x_p, "negative": x_n}
        
        elif self.mode == "pair":
            a = x
            a_label = author

            # Choose another idx at random
            idx2 = np.random.choice(np.arange(len(self.df)))
            b = self.load_audio_tensor(idx2)
            b_label = self.id_to_author[idx2]

            return {"a": a, "b": b, "a_label": a_label, "b_label": b_label}

        else:
            raise NotImplementedError(f"Mode {self.mode} not implemented")


In [75]:
TEXT_FEATURES = 1024
SPEECH_FEATURES = 256

dataset = InTheWildDataset(
    root_dir="../datasets/InTheWild",
    metadata_file="wild_transcription_meta.json",
    include_spoofs=True,
    bonafide_label="bona-fide",
    filename_col="file",
    transcription_col='content',
    sampling_rate=16000,
    max_duration=4,
    split="train",
    text_tokenizer_name=TEXT_MODEL_NAME,
    config="/home/infres/amathur-23/DADA/src/SpeechCLR/configs/data/inthewild_toy.yaml", 
    mode="classification",
)

loader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=True)

/home/infres/amathur-23/DADA/dada/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [76]:
x = next(iter(loader))

In [77]:
x

{'x': tensor([[-1.7785e-04, -3.3424e-04, -9.4488e-05,  ..., -3.9154e-02,
          -3.5004e-02, -2.8067e-02],
         [-4.8968e-04, -1.8645e-04, -1.6061e-03,  ..., -1.9415e-02,
           7.7594e-03,  2.0707e-02],
         [ 1.5939e-04,  1.6132e-03,  3.0111e-03,  ..., -2.5401e-03,
          -2.3908e-03, -2.1925e-03],
         [-4.9476e-07,  4.5353e-05,  7.9203e-05,  ...,  6.8018e-02,
           6.9764e-02,  6.9907e-02]]),
 'label': tensor([1, 0, 0, 0]),
 'author': ['Bernie Sanders',
  'Alec Guinness',
  'Donald Trump',
  'Bernie Sanders'],
 'transcription': {'input_ids': tensor([[    1,   379,   261,   379,   459,  8254,   260,     2,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0, 

In [82]:
class MidFuse(nn.Module):
    def __init__(self, text_model, speech_model, text_features, speech_features):
        super(MidFuse, self).__init__()
        self.text_model = text_model
        self.speech_model = speech_model
        self.text_features = text_features
        self.speech_features = speech_features


        self.classifier = nn.Sequential(
            nn.Linear(text_features + speech_features, 512),
            nn.Linear(512, 256),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def train(self):
        self.text_model.eval()
        self.speech_model.eval()
        self.classifier.train()

    def trainable_parameters(self):
        return self.classifier.parameters()
    
    def eval(self):
        self.text_model.eval()
        self.speech_model.eval()
        self.classifier.eval()

    def forward(self, text_input, speech_input):
        text_features = self.text_model(input_ids = text_input['input_ids'], attention_mask = text_input['attention_mask'])
        speech_features = self.speech_model(speech_input, mode='classification')
        features = torch.cat([text_features, speech_features], dim=1)
        x = self.classifier(features)
        return x

In [83]:
mid_fuse_model = MidFuse(text_model, speech_model, TEXT_FEATURES, SPEECH_FEATURES)

In [84]:
mid_fuse_model

MidFuse(
  (text_model): AuthorshipLLM(
    (model): DebertaV2Model(
      (embeddings): DebertaV2Embeddings(
        (word_embeddings): Embedding(128100, 1024, padding_idx=0)
        (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
        (dropout): StableDropout()
      )
      (encoder): DebertaV2Encoder(
        (layer): ModuleList(
          (0-23): 24 x DebertaV2Layer(
            (attention): DebertaV2Attention(
              (self): DisentangledSelfAttention(
                (query_proj): Linear(in_features=1024, out_features=1024, bias=True)
                (key_proj): Linear(in_features=1024, out_features=1024, bias=True)
                (value_proj): Linear(in_features=1024, out_features=1024, bias=True)
                (pos_dropout): StableDropout()
                (dropout): StableDropout()
              )
              (output): DebertaV2SelfOutput(
                (dense): Linear(in_features=1024, out_features=1024, bias=True)
                (LayerNo

In [ ]:
from tqdm import tqdm

optimizer = torch.optim.Adam(mid_fuse_model.trainable_parameters(), lr=1e-4)
criterion = nn.BCELoss()

mid_fuse_model.train()
mid_fuse_model.to('cuda')

for epoch in range(5):
    losses = []
    for i, batch in enumerate(loader):
        text = batch['transcription']
        speech = batch['x']
        label = batch['label'].squeeze().float().to('cuda')

        text = {k: v.to('cuda') for k, v in text.items()}
        batch['x'] = batch['x'].to('cuda')

        optimizer.zero_grad()
        output = mid_fuse_model(text_input = text, speech_input = batch).squeeze()
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())

        # if i % 100 == 0:
        print(f"\r Epoch {epoch + 1} [{i+1}/{len(loader)}] Loss: {np.mean(losses)}", end="")
    print()


 Epoch 1 [285/285] Loss: 0.50141384035610316
 Epoch 2 [120/285] Loss: 0.46988484381387636